# Análise de Texto com Elasticsearch

## Importando o pacote cliente do Elasticsearch

Para instalar o pacote do Elasticsearch, use:

```
pip install elasticsearch
```

In [1]:
import elasticsearch

ES_URL = 'https://localhost:9200'
ES_USER = 'elastic'
ES_PASS = 'elastic123'

client = elasticsearch.Elasticsearch(
    ES_URL,
    basic_auth=(ES_USER, ES_PASS),
    verify_certs=False
)

/home/felipefg/.pyenv/versions/3.9.1/envs/infnet/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [2]:
# Desligando warning para facilitar nossa vida
import urllib3
urllib3.disable_warnings()

In [3]:
dict(client.info())

{'name': 'df5478436d68',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'uQ077_TxQeGa8WqnucK7qw',
 'version': {'number': '8.2.2',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '9876968ef3c745186b94fdabd4483e01499224ef',
  'build_date': '2022-05-25T15:47:06.259735307Z',
  'build_snapshot': False,
  'lucene_version': '9.1.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

# Testando Configurações de análise

In [4]:
resp = client.indices.analyze(
        analyzer="whitespace",
        text="Eu não quero um copo d'água."
)
dict(resp)

{'tokens': [{'token': 'Eu',
   'start_offset': 0,
   'end_offset': 2,
   'type': 'word',
   'position': 0},
  {'token': 'não',
   'start_offset': 3,
   'end_offset': 6,
   'type': 'word',
   'position': 1},
  {'token': 'quero',
   'start_offset': 7,
   'end_offset': 12,
   'type': 'word',
   'position': 2},
  {'token': 'um',
   'start_offset': 13,
   'end_offset': 15,
   'type': 'word',
   'position': 3},
  {'token': 'copo',
   'start_offset': 16,
   'end_offset': 20,
   'type': 'word',
   'position': 4},
  {'token': "d'água.",
   'start_offset': 21,
   'end_offset': 28,
   'type': 'word',
   'position': 5}]}

Mesmo exemplo, mas deixando o resultado mais legível:

In [5]:
analyzed = client.indices.analyze(
        analyzer="whitespace",
        text="Eu não quero um copo d'água."
)

print([x["token"] for x in analyzed['tokens']])

['Eu', 'não', 'quero', 'um', 'copo', "d'água."]


## O analisador `standard`

In [6]:
analyzed = client.indices.analyze(
    analyzer="standard",
    text="Eu não quero um copo d'água."
)

print([x["token"] for x in analyzed['tokens']])

['eu', 'não', 'quero', 'um', 'copo', "d'água"]


## Especificando os componentes individualmente

Em vez de especificar um analisador pré-configurado, podemos especificar os componentes individualmente, como abaixo

In [7]:
analyzed = client.indices.analyze(
    tokenizer="standard",
    filter=['asciifolding', 'lowercase'],
    text="Eu não quero um copo d'água.",
)

print([x["token"] for x in analyzed['tokens']])

['eu', 'nao', 'quero', 'um', 'copo', "d'agua"]


O filtro `elision` é pensado para o Francês, e retira os prefixos transformando, por exemplo, `l'avion` em `avion`.

In [8]:
analyzed = client.indices.analyze(
    tokenizer="standard",
    filter=['asciifolding', 'lowercase', 'elision'],
    text="Eu não quero um copo d'água.",
)

print([x["token"] for x in analyzed['tokens']])

['eu', 'nao', 'quero', 'um', 'copo', 'agua']


## Criando um Analyzer customizado

In [9]:
index_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "meu_analisador": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "asciifolding",
                        "lowercase",
                        "elision",
                    ]
                }
            }
        }
    }
}

In [10]:
if client.indices.exists(index="meu_indice"):
    client.indices.delete(index="meu_indice")
client.indices.create(index="meu_indice", **index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'meu_indice'})

In [11]:
analyzed = client.indices.analyze(
    index="meu_indice",
    analyzer="meu_analisador",
    text="Eu não quero um copo d'água.",
)

print([x["token"] for x in analyzed['tokens']])

['eu', 'nao', 'quero', 'um', 'copo', 'agua']


## Criando um Analyzer com filtros customizados

In [12]:
index_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "meu_analisador": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "elision",
                        "stop_pt",
                        "asciifolding",
                        "stop_custom",
                    ]
                }
            },
            "filter": {
                "stop_pt": {
                    "type": "stop",
                    "stopwords": "_portuguese_",
                },
                "stop_custom": {
                    "type": "stop",
                    "stopwords": ['agua'],
                },
            }
        }
    }
}

In [13]:
if client.indices.exists(index="meu_indice"):
    client.indices.delete(index="meu_indice")
client.indices.create(index="meu_indice", **index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'meu_indice'})

In [14]:
analyzed = client.indices.analyze(
    index="meu_indice",
    analyzer="meu_analisador",
    text="Eu não quero um copo d'água.",
)

print([x["token"] for x in analyzed['tokens']])

['quero', 'copo']


In [15]:
index_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "meu_analisador": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "elision",
                        "radiciacao",
                        #"stop_pt",
                        #"asciifolding",
                        #"stop_custom",
                    ]
                }
            },
            "filter": {
                "stop_pt": {
                    "type": "stop",
                    "stopwords": "_portuguese_",
                },
                "stop_custom": {
                    "type": "stop",
                    "stopwords": ['agua'],
                },
                "radiciacao": {
                    "type": "snowball",
                    "language": "Portuguese"
                }
            }
        }
    }
}

In [16]:
if client.indices.exists(index="meu_indice"):
    client.indices.delete(index="meu_indice")
client.indices.create(index="meu_indice", **index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'meu_indice'})

In [17]:
analyzed = client.indices.analyze(
    index="meu_indice",
    analyzer="meu_analisador",
    text="Eu não quero um copo d'água branca branco.",
)

print([x["token"] for x in analyzed['tokens']])

['eu', 'nã', 'quer', 'um', 'cop', 'águ', 'branc', 'branc']
